In [1]:
# Import Pandas
import pandas as pd
import psycopg2
from config import endpoint, username, password
from sqlalchemy import create_engine

In [2]:
# Read CSVs for State Data
state_pop_df = pd.read_csv("data/state_populations_cleaned.csv")
state_vacc_df = pd.read_csv("data/state_covid_vaccinations_cleaned.csv")

In [3]:
# State Populations
state_pop_df.head()

,state,population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223


In [4]:
# State Vaccinations
state_vacc_df.head()

,state,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
0,Alaska,154325,71907,21096,9829,59449,8126,12352,1688
1,Alabama,493125,165919,10057,3384,144429,2946,21345,435
2,Arkansas,361550,173312,11981,5743,144792,4798,26541,879
3,Arizona,797550,303899,10957,4175,262120,3601,41613,572
4,California,4379500,1633875,11084,4135,1335886,3381,293834,744


In [5]:
# State Data Merged
state_merged_df = pd.merge(state_pop_df, state_vacc_df, how="left", on="state")
state_merged_df.head()

,state,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
0,Alabama,4903185,493125,165919,10057,3384,144429,2946,21345,435
1,Alaska,731545,154325,71907,21096,9829,59449,8126,12352,1688
2,Arizona,7278717,797550,303899,10957,4175,262120,3601,41613,572
3,Arkansas,3017804,361550,173312,11981,5743,144792,4798,26541,879
4,California,39512223,4379500,1633875,11084,4135,1335886,3381,293834,744


In [6]:
# Set State as the Index
state_info_df = state_merged_df.set_index("state") 
state_info_df.head()

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
state,,,,,,,,,
Alabama,4903185,493125,165919,10057,3384,144429,2946,21345,435
Alaska,731545,154325,71907,21096,9829,59449,8126,12352,1688
Arizona,7278717,797550,303899,10957,4175,262120,3601,41613,572
Arkansas,3017804,361550,173312,11981,5743,144792,4798,26541,879
California,39512223,4379500,1633875,11084,4135,1335886,3381,293834,744


In [7]:
# Add Empty Column for New Calculation
state_info_df["percent_vaccinated"] = ""
state_info_df.head(10)

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,percent_vaccinated
state,,,,,,,,,,
Alabama,4903185,493125,165919,10057,3384,144429,2946,21345,435,
Alaska,731545,154325,71907,21096,9829,59449,8126,12352,1688,
Arizona,7278717,797550,303899,10957,4175,262120,3601,41613,572,
Arkansas,3017804,361550,173312,11981,5743,144792,4798,26541,879,
California,39512223,4379500,1633875,11084,4135,1335886,3381,293834,744,
Colorado,5758736,657250,353194,11413,6133,296816,5154,54474,946,
Connecticut,3565287,435075,264707,12203,7425,230913,6477,28602,802,
Delaware,973764,106600,49369,10947,5070,41900,4303,7355,755,
District of Columbia,705749,75175,55108,10652,7808,43377,6146,11685,1656,


In [8]:
# Calcuation for percent vaccinated by state
for index, row in state_info_df.iterrows():
    population = row["population"]
    vaccinated = row["total_administered"]
    percent_vaccinated = round((vaccinated / population) * 100, 2)
    
    state_info_df.at[index, "percent_vaccinated"] = percent_vaccinated

In [9]:
# Check the data types
state_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, Alabama to Wyoming
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   population                        51 non-null     int64 
 1   total_distributed                 51 non-null     int64 
 2   total_administered                51 non-null     int64 
 3   distributed_per_100K              51 non-null     int64 
 4   administered_per_100K             51 non-null     int64 
 5   people_with_1plus_doses           51 non-null     int64 
 6   people_with_1plus_doses_per_100K  51 non-null     int64 
 7   people_with_2_doses               51 non-null     int64 
 8   people_with_2_doses_per_100K      51 non-null     int64 
 9   percent_vaccinated                51 non-null     object
dtypes: int64(9), object(1)
memory usage: 6.9+ KB


In [10]:
# Convert percent vaccinated into a float
state_info_df = state_info_df.astype({"percent_vaccinated": 'float64'})
state_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, Alabama to Wyoming
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   population                        51 non-null     int64  
 1   total_distributed                 51 non-null     int64  
 2   total_administered                51 non-null     int64  
 3   distributed_per_100K              51 non-null     int64  
 4   administered_per_100K             51 non-null     int64  
 5   people_with_1plus_doses           51 non-null     int64  
 6   people_with_1plus_doses_per_100K  51 non-null     int64  
 7   people_with_2_doses               51 non-null     int64  
 8   people_with_2_doses_per_100K      51 non-null     int64  
 9   percent_vaccinated                51 non-null     float64
dtypes: float64(1), int64(9)
memory usage: 6.9+ KB


In [11]:
# Export as json
state_info_df.to_json("data/state_data.json")

In [12]:
# Connect to AWS Database instance 
engine = create_engine(f'postgresql://uscovid:{password}@{endpoint}/us_covid_db')
connection = engine.connect()

In [13]:
# Add to Database and replace if the table already exists
state_info_df.to_sql('master_table', index=True, if_exists='replace', con=connection)

## NIH AWS Data / NYT

In [63]:
# Read NYT / NIH json 
nytimes_nih_covid_df = pd.read_json("data/daily_covid_stats_by_state_nyt.json", lines=True)

In [64]:
nyt_state_grouped = nytimes_nih_covid_df.groupby(['state', 'date'])
total_cases_by_date_df = pd.DataFrame(nyt_state_grouped["cases"].sum())

In [103]:
total_cases_by_date_df

cases
state   date             
Alabama 2020-03-13      6
        2020-03-14     12
        2020-03-15     23
        2020-03-16     29
        2020-03-17     39
...                   ...
Wyoming 2021-01-21  50424
        2021-01-22  50583
        2021-01-23  50583
        2021-01-24  50898
        2021-01-25  51062

[18109 rows x 1 columns]

In [65]:
total_cases_by_date_df.to_json("data/total_cases_by_state_and_date.json")

In [ ]:
# Add data to database 
nytimes_nih_covid_df.to_sql('nyt_table', index=False, if_exists='replace', con=connection)

In [89]:
latest_nyt_stateData = nytimes_nih_covid_df[nytimes_nih_covid_df['date'] == '2021-01-25']

In [90]:
latest_data_merged = pd.merge(state_info_df, latest_nyt_stateData, on="state")

In [91]:
latest_data_merged = latest_data_merged[['state', 'population', 'cases', 'deaths', 'total_distributed', 'total_administered', 'percent_vaccinated' ]]

In [92]:
latest_data_merged["percent_infected_to_date"] = ""

In [93]:
latest_data_merged

,state,population,cases,deaths,total_distributed,total_administered,percent_vaccinated,percent_infected_to_date
0,Alabama,4903185,443009,6662,493125,165919,3.38,
1,Alaska,731545,53061,250,154325,71907,9.83,
2,Arizona,7278717,728231,12239,797550,303899,4.18,
3,Arkansas,3017804,284702,4650,361550,173312,5.74,
4,California,39512223,3212488,37499,4379500,1633875,4.14,
5,Colorado,5758736,389227,5602,657250,353194,6.13,
6,Connecticut,3565287,243632,6911,435075,264707,7.42,
7,Delaware,973764,75191,1049,106600,49369,5.07,
8,District of Columbia,705749,35505,879,75175,55108,7.81,
9,Florida,21477737,1658161,25445,2546050,1239975,5.77,


In [94]:
# Calcuation for percent infected by state
for index, row in latest_data_merged.iterrows():
    population = row["population"]
    deaths = row["deaths"]
    infected = row["cases"]
    est_population_new = population - deaths 
    
    percent_infected = round((infected / est_population_new) * 100, 2)
    
    latest_data_merged.at[index, "percent_infected_to_date"] = percent_infected

In [95]:
latest_data_merged["est_percent_immune"] = ""

In [96]:
latest_data_merged

,state,population,cases,deaths,total_distributed,total_administered,percent_vaccinated,percent_infected_to_date,est_percent_immune
0,Alabama,4903185,443009,6662,493125,165919,3.38,9.05,
1,Alaska,731545,53061,250,154325,71907,9.83,7.26,
2,Arizona,7278717,728231,12239,797550,303899,4.18,10.02,
3,Arkansas,3017804,284702,4650,361550,173312,5.74,9.45,
4,California,39512223,3212488,37499,4379500,1633875,4.14,8.14,
5,Colorado,5758736,389227,5602,657250,353194,6.13,6.77,
6,Connecticut,3565287,243632,6911,435075,264707,7.42,6.85,
7,Delaware,973764,75191,1049,106600,49369,5.07,7.73,
8,District of Columbia,705749,35505,879,75175,55108,7.81,5.04,
9,Florida,21477737,1658161,25445,2546050,1239975,5.77,7.73,


In [97]:
# Calcuation for percent infected by state
for index, row in latest_data_merged.iterrows():
    population = row["population"]
    infected = row["cases"]
    deaths = row["deaths"]
    vaccinated = row["total_administered"]
    
    est_total_immune = infected + vaccinated
    est_population = population - deaths
    
    percent_infected = round((est_total_immune / est_population) * 100, 2)
    
    latest_data_merged.at[index, "est_percent_immune"] = percent_infected

In [98]:
latest_data_merged

,state,population,cases,deaths,total_distributed,total_administered,percent_vaccinated,percent_infected_to_date,est_percent_immune
0,Alabama,4903185,443009,6662,493125,165919,3.38,9.05,12.44
1,Alaska,731545,53061,250,154325,71907,9.83,7.26,17.09
2,Arizona,7278717,728231,12239,797550,303899,4.18,10.02,14.2
3,Arkansas,3017804,284702,4650,361550,173312,5.74,9.45,15.2
4,California,39512223,3212488,37499,4379500,1633875,4.14,8.14,12.28
5,Colorado,5758736,389227,5602,657250,353194,6.13,6.77,12.9
6,Connecticut,3565287,243632,6911,435075,264707,7.42,6.85,14.29
7,Delaware,973764,75191,1049,106600,49369,5.07,7.73,12.81
8,District of Columbia,705749,35505,879,75175,55108,7.81,5.04,12.86
9,Florida,21477737,1658161,25445,2546050,1239975,5.77,7.73,13.51


In [99]:
latest_data_merged.to_sql('state_est_immune', index=False, if_exists='replace', con=connection)

In [100]:
latest_data_merged_json = latest_data_merged.set_index("state")

In [101]:
latest_data_merged_json

,population,cases,deaths,total_distributed,total_administered,percent_vaccinated,percent_infected_to_date,est_percent_immune
state,,,,,,,,
Alabama,4903185,443009,6662,493125,165919,3.38,9.05,12.44
Alaska,731545,53061,250,154325,71907,9.83,7.26,17.09
Arizona,7278717,728231,12239,797550,303899,4.18,10.02,14.2
Arkansas,3017804,284702,4650,361550,173312,5.74,9.45,15.2
California,39512223,3212488,37499,4379500,1633875,4.14,8.14,12.28
Colorado,5758736,389227,5602,657250,353194,6.13,6.77,12.9
Connecticut,3565287,243632,6911,435075,264707,7.42,6.85,14.29
Delaware,973764,75191,1049,106600,49369,5.07,7.73,12.81
District of Columbia,705749,35505,879,75175,55108,7.81,5.04,12.86


In [102]:
latest_data_merged_json.to_json("data/latest_est_immune.json")